# Additional datasets from other studies
This notebook describes the methods to access the genomes from other studies used in the analysis

In [1]:
import pandas as pd
import json
from pathlib import Path
import tarfile
import gzip
import shutil
import requests, json

In [ ]:
# Set up paths
## Path to BGCflow directory
external_data_path = Path("/datadrive/bgcflow/data/external")

## Path to tables from other studies
external_table_path = Path("../tables/other_studies")
external_table_path.mkdir(parents=True, exist_ok=True)

## Generate a config template for BGCflow
df_bgcflow_template = pd.DataFrame(columns=["source","organism","genus","species","strain","closest_placement_reference"])

## Dataset from Bickhart et al (464 genomes)
The dataset can be accessed from https://zenodo.org/record/5138306/files/hifi_das.bin3c.bins.tar.gz. The dataset are then downloaded using wget:

In [ ]:
# MAG set for the Bickhart paper (464)
! wget -P ../data https://zenodo.org/record/5138306/files/hifi_das.bin3c.bins.tar.gz?download=1 -nc
! mv ../data/hifi_das.bin3c.bins.tar.gz\?download\=1 ../data/hifi_das.bin3c.bins.tar.gz
! tar -xvzf ../data/hifi_das.bin3c.bins.tar.gz -C /target/directory
tar = tarfile.open("sample.tar.gz")
tar.extractall()
tar.close()

The metadata for the genomes are then cleaned to create a list of genomes that will be used in the analysis:

In [2]:
# Download supplementary materials from the paper
! wget -P ../data https://static-content.springer.com/esm/art%3A10.1038%2Fs41587-021-01130-z/MediaObjects/41587_2021_1130_MOESM3_ESM.xlsx -nc

# Clean Up
df_bickhart = pd.read_excel("../data/41587_2021_1130_MOESM3_ESM.xlsx")
df_bickhart = pd.concat([df_bgcflow_template, df_bickhart])
df_bickhart.insert(0, 'genome_id', df_bickhart.loc[:, "MAG"])
df_bickhart = df_bickhart.drop(columns=["MAG"])
df_bickhart.loc[:, "source"] = "custom"
df_bickhart.to_csv(external_table_path / "df_bickhart.csv", index=None)

File ‘../data/41587_2021_1130_MOESM3_ESM.xlsx’ already there; not retrieving.



The selected genomes are then copied to the BGCflow directory for downstream analyses:

In [6]:
# Set Up Paths
bickhart_raw_path = Path("../data/b3c_hifi_dastool/")
bickhart_data_path = external_data_path / "Bickhart_et_al"
bickhart_data_path.mkdir(parents=True, exist_ok=True)

# copy files to target dir
for i in df_bickhart.loc[:, "genome_id"]:
    item = bickhart_raw_path / f"flye4.das_DASTool_bins/{i}.contigs.fa.gz"
    dest = bickhart_data_path / f"{i}.fna"
    if dest.is_file():
        pass
    else:
        if item.is_file():
            with gzip.open(item, 'rb') as f_in:
                with open(dest, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
        else:
            item = bickhart_raw_path / f"flye4.das_DASTool_bins/{i}.contigs.fa"
            assert(item.is_file())
            shutil.copy(item, dest)

# Dataset from Liu et al (557 genomes)
The dataset was kindly provided by the authors here: https://www.dropbox.com/sh/qj4aginbflqmxhq/AAB5INvLNKrGiBtjpRbteYJ7a. The dataset are then downloaded using wget:

In [ ]:
# MAG set for the Liu paper (557)
! wget -P ../data https://www.dropbox.com/sh/qj4aginbflqmxhq/AAB5INvLNKrGiBtjpRbteYJ7a?dl=0 -nc
! mv ../data/AAB5INvLNKrGiBtjpRbteYJ7a\?dl\=0 ../data/Liu_et_al.zip
! unzip ../data/Liu_et_al.zip

The metadata for the genomes are then cleaned to create a list of genomes that will be used in the analysis:

In [49]:
# Download supplementary materials from the paper
! wget -P ../data https://static-content.springer.com/esm/art%3A10.1186%2Fs40168-021-01155-1/MediaObjects/40168_2021_1155_MOESM2_ESM.xlsx -nc

# Clean Up
df_liu = pd.read_excel("../data/40168_2021_1155_MOESM2_ESM.xlsx", sheet_name=2, skiprows=0)
df_liu = pd.concat([df_bgcflow_template, df_liu])
df_liu.insert(0, 'genome_id', df_liu.loc[:, "MAGs"])
df_liu = df_liu.drop(columns=["MAGs"])
df_liu.loc[:, "source"] = "custom"
df_liu.to_csv(external_table_path / "df_liu.csv", index=None)
# ! for f in *.fasta; do mv -- "$f" "${f%.fasta}.fna"; done

File ‘../data/40168_2021_1155_MOESM2_ESM.xlsx’ already there; not retrieving.



# Dataset from Christoph et al (73 genomes)
The dataset from Christoph et al are publicly available from NCBI: https://www.ncbi.nlm.nih.gov/bioproject/?term=PRJNA449266. The assembly details file: `PRJNA449266_AssemblyDetails.txt` were downloaded, containing the assembly accessions of the study, which then can be downloaded using ncbi-genome-download.

HQ-MAGs are selected from the dataset with this definition:
>High-quality draft' will indicate that a SAG or MAG is >90% complete with less than 5% contamination

The metadata for the genomes are then cleaned to create a list of genomes that will be used in the analysis:

In [76]:
# Download supplementary materials from the paper
! wget -P ../data https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-018-0207-y/MediaObjects/41586_2018_207_MOESM3_ESM.xlsx -nc
df_christoph = pd.read_excel("../data/41586_2018_207_MOESM3_ESM.xlsx", sheet_name=0, skiprows=0)

# filter for HQ
a = df_christoph.loc[:, "CheckM Completeness %"] > 90
b = df_christoph.loc[:, "CheckM Contamination %"] < 5
df_christoph = df_christoph[a & b]

# Get NCBI accession
df_christoph.loc[:, "ncbi_mapping"] = [i.split("_", 1)[-1].replace("_", " ") for i in df_christoph.loc[:, "Genome"]]
need_cleaning = df_christoph.loc[:, "ncbi_mapping"].str.startswith("unk")
df_christoph.loc[df_christoph[need_cleaning].index, "ncbi_mapping"] = [i.split(" ")[-1] for i in df_christoph[need_cleaning].loc[:, "ncbi_mapping"]]
df_christoph_ncbi = pd.read_csv("../data/PRJNA449266_AssemblyDetails.txt", sep="\t", skiprows=1, index_col=False)
df_christoph_ncbi_subset = df_christoph_ncbi[df_christoph_ncbi.loc[:, "Isolate"].isin(df_christoph.loc[:, "ncbi_mapping"])]
df_christoph = df_christoph.merge(df_christoph_ncbi_subset, how="left", left_on="ncbi_mapping", right_on="Isolate")
df_christoph = pd.concat([df_bgcflow_template, df_christoph])
df_christoph.insert(0, 'genome_id', df_christoph.loc[:, "# Assembly"])
df_christoph = df_christoph.drop(columns=["# Assembly"])
df_christoph.loc[:, "source"] = "ncbi"
df_christoph.to_csv("../tables/other_studies/df_christoph.csv", index=None)

File ‘../data/41586_2018_207_MOESM3_ESM.xlsx’ already there; not retrieving.



Some of these genomes are already used as GTDB reference. These genomes are discarded in the GTDB-tk classify wf as it will return an error:

In [3]:
pd.read_csv("../tables/gtdbtk.warnings.log", skiprows=1, header=None, sep=" ").loc[:, 3].to_csv("../tables/filter_gtdb.txt", header=None, index=None)

# Dataset from Sharrar et al (374 genomes)

The dataset was kindly provided by the authors here: https://figshare.com/ndownloader/files/18105260. The dataset are then downloaded using wget:

In [ ]:
! wget -P ../data https://figshare.com/ndownloader/files/18105260 -nc
! mv ../data/18105260 ../data/1334_genomes.tar.gz # then untar

The metadata for the genomes are then cleaned to create a list of genomes that will be used in the analysis:

In [53]:
# Download supplementary materials from the paper
! wget -P ../data https://journals.asm.org/doi/suppl/10.1128/mBio.00416-20/suppl_file/mbio.00416-20-st002.xlsx -nc

# Clean Up
df_sharrar = pd.read_excel("../data/mbio.00416-20-st002.xlsx", sheet_name=0, skiprows=0)
df_sharrar = df_sharrar[df_sharrar.loc[:, "MAG draft quality"] == "High"]
df_sharrar = pd.concat([df_bgcflow_template, df_sharrar])
df_sharrar.insert(0, 'genome_id', df_sharrar.loc[:, "Genome name"])
df_sharrar = df_sharrar.drop(columns=["Genome name"])
df_sharrar.loc[:, "source"] = "custom"
df_sharrar.to_csv("../tables/other_studies/df_sharrar.csv", index=None)

The selected genomes are then copied to the BGCflow directory for downstream analyses:

In [85]:
# copy files to target dir
sharrar_raw_path = Path("../data/1334_genomes")
sharrar_data_path = external_data_path / "Sharrar_et_al"
sharrar_data_path.mkdir(parents=True, exist_ok=True)
for i in df_sharrar.loc[:, "genome_id"]:
    item = sharrar_raw_path / f"{i}.contigs.fa"
    dest = sharrar_data_path / f"{i}.fna"
    shutil.copy(item, dest)

# Downloading dataset from Chen et al
The dataset for Chen et al are available from: https://api.mg-rast.org/project/mgp81948?verbosity=full. The genomes can then be fetched using MG-RAST API:

In [ ]:
def get_api(api_url, output):
    response = requests.get(api_url)
    project_metadata = response.json()
    with open(output, 'w') as outfile:
        json.dump(project_metadata, outfile)
    return None

get_api('https://api.mg-rast.org/project/mgp81948?verbosity=full', '../data/mgp81948.json')

In [ ]:
with open('../data/mgp81948.json', "r") as f:
    study_metadata = json.load(f)
    ids = [i["metagenome_id"] for i in study_metadata["metagenomes"]]
    for mg_id in ids:
        url = f"https://api.mg-rast.org/download/{mg_id}?file=299.1"
        ! wget -O ../data/chen/{mg_id}.fna {url}

The metadata for the genomes cam be accessed from https://www.frontiersin.org/articles/10.3389/fmicb.2020.01950/full#supplementary-material. The metadata are then cleaned to create a list of genomes that will be used in the analysis:

In [82]:
# Download supplementary materials from the paper
df_chen_metadata = pd.read_excel("../data/Table_1_Discovery of an Abundance of Biosynthetic Gene Clusters in Shark Bay Microbial Mats.XLSX", sheet_name=0, skiprows=5)

with open("../data/mgp81948.json", "r") as file:
    chen_metadata = json.load(file)
df_chen = pd.DataFrame.from_dict([i for i in chen_metadata["metagenomes"]])
df_chen = pd.concat([df_bgcflow_template, df_chen])
df_chen.insert(0, 'genome_id', df_chen.loc[:, "metagenome_id"])
df_chen = df_chen.drop(columns=["metagenome_id"])
df_chen.loc[:, "source"] = "custom"
df_chen.to_csv("../tables/other_studies/df_chen.csv", index=None)